In [1]:
import numpy as np
import os


import random
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy import interpolate
import itertools
import astropy.units as u
import astropy.constants as const
import scipy.stats

from matplotlib.patches import Patch

In [2]:
import sys

from s1s2_0311 import *
from read import *
from recoil_1223 import *

from plot_binned_detection_1d2d_0526 import *

In [3]:
file_name = 'atmhepdsnb_Xe_DT_optimized_consecutive_thrd0_numeNa_S1S2binning.csv'
file_name

'atmhepdsnb_Xe_DT_optimized_consecutive_thrd0_numeNa_S1S2binning.csv'

In [4]:
etaijk_pd = pd.read_csv('atmhepdsnb_Xe_ETAijk_optimized_consecutive_thrd0_S1S2binning.csv', index_col = 0)
etaijk_pd

,null_sum,alter_sum
atm_CJPL_high,0.002876,0.008038
atm_CJPL_low,0.002876,0.008039
atm_Kamioka_high,0.003484,0.010487
atm_Kamioka_low,0.003487,0.010490
atm_LNGS_high,0.004901,0.015588
atm_LNGS_low,0.004910,0.015597
atm_SURF_high,0.006115,0.022153
atm_SURF_low,0.006126,0.022164
atm_SNOlab_high,0.006114,0.022294
atm_SNOlab_low,0.006124,0.022305


In [6]:
output_type = ''

In [7]:
signal_pcles  =['atm', 'hep', 'dsnb']#['dsnb'] #['atm', 'hep', 'dsnb']

In [8]:
detectors = ['CJPL', 'Kamioka', 'LNGS', 'SURF', 'SNOlab']
alphas_powers = [(0.001, 0.9), (0.01, 0.9),  (0.1, 0.9),
                (0.001, 0.95), (0.01, 0.95), (0.1, 0.95),
                (0.001, 0.8), (0.01, 0.8), (0.1, 0.8)]
metallicities = ['high', 'low']

In [9]:
if output_type == 'modify':
    result_pd = pd.read_csv(file_name, index_col = 0)
else:
    result_pd = pd.DataFrame(index = [str(alpha_power) for alpha_power in alphas_powers],
                         columns=[signal_pcle+'_'+detector+'_'+metallicity for signal_pcle in signal_pcles
                                  for detector in detectors 
                                  for  metallicity in metallicities])


In [10]:
result_pd

,atm_CJPL_high,atm_CJPL_low,atm_Kamioka_high,atm_Kamioka_low,atm_LNGS_high,atm_LNGS_low,atm_SURF_high,atm_SURF_low,atm_SNOlab_high,atm_SNOlab_low,...,dsnb_CJPL_high,dsnb_CJPL_low,dsnb_Kamioka_high,dsnb_Kamioka_low,dsnb_LNGS_high,dsnb_LNGS_low,dsnb_SURF_high,dsnb_SURF_low,dsnb_SNOlab_high,dsnb_SNOlab_low
"(0.001, 0.9)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.01, 0.9)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.1, 0.9)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.001, 0.95)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.01, 0.95)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.1, 0.95)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.001, 0.8)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.01, 0.8)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(0.1, 0.8)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
for signal_pcle in ['dsnb']:
    if signal_pcle == 'dsnb':
        exposures = [int(i) for i in np.logspace(np.log10(4000), np.log10(40000), 501)]

        
    elif signal_pcle == 'hep':
        exposures = [int(i) for i in np.logspace(np.log10(1000), np.log10(8000), 101)]

    elif signal_pcle == 'atm':
        exposures = [int(i) for i in np.logspace(np.log10(200), np.log10(7000), 101)]
    print('scan:', exposures)
    for detector in detectors[1:]:   
        for m, metallicity in enumerate(metallicities):

            null_sum_ijkappa = etaijk_pd['null_sum'][signal_pcle + '_'+detector+'_'+metallicity]
            alter_sum_ijkappa = etaijk_pd['alter_sum'][signal_pcle + '_'+detector+'_'+metallicity]
            print(metallicity, 'null_sum_ij = ', null_sum_ijkappa, 
                  '\nalter_sum_ij = ', alter_sum_ijkappa)

            for alpha_power in alphas_powers:
                dict_key = str(alpha_power)
                alpha, power = alpha_power
                DT, Ns_alpha, powers, Gs, test_exposures = get_nume_DT_alphabeta_G_method2(
                    exposures, null_sum_ijkappa, alter_sum_ijkappa, alpha, power, 
                        print_check = False, plot = False)
                result_pd[signal_pcle+'_'+detector+'_'+metallicity][str(alpha_power)] = DT

                print(signal_pcle, detector, metallicity, 'alpha = ', alpha, 'power = ', power, 'DT = ',DT)




scan: [4000, 4018, 4037, 4055, 4074, 4093, 4112, 4131, 4150, 4169, 4188, 4207, 4227, 4246, 4266, 4286, 4305, 4325, 4345, 4365, 4385, 4406, 4426, 4446, 4467, 4488, 4508, 4529, 4550, 4571, 4592, 4613, 4635, 4656, 4677, 4699, 4721, 4743, 4764, 4786, 4809, 4831, 4853, 4875, 4898, 4921, 4943, 4966, 4989, 5012, 5035, 5058, 5082, 5105, 5129, 5152, 5176, 5200, 5224, 5248, 5273, 5297, 5321, 5346, 5371, 5395, 5420, 5445, 5470, 5496, 5521, 5547, 5572, 5598, 5624, 5650, 5676, 5702, 5728, 5755, 5781, 5808, 5835, 5862, 5889, 5916, 5943, 5971, 5998, 6026, 6054, 6082, 6110, 6138, 6166, 6195, 6223, 6252, 6281, 6310, 6339, 6368, 6398, 6427, 6457, 6487, 6517, 6547, 6577, 6607, 6638, 6668, 6699, 6730, 6761, 6792, 6824, 6855, 6887, 6919, 6951, 6983, 7015, 7047, 7080, 7113, 7145, 7178, 7212, 7245, 7278, 7312, 7346, 7380, 7414, 7448, 7482, 7517, 7551, 7586, 7621, 7657, 7692, 7727, 7763, 7799, 7835, 7871, 7907, 7944, 7981, 8017, 8054, 8092, 8129, 8166, 8204, 8242, 8280, 8318, 8357, 8395, 8434, 8473, 8512, 855

dsnb SNOlab low alpha =  0.01 power =  0.9 DT =  20497.366413778964
dsnb SNOlab low alpha =  0.1 power =  0.9 DT =  10589.488967583766
dsnb SNOlab low alpha =  0.001 power =  0.95 DT =  35297.46987951807
dsnb SNOlab low alpha =  0.01 power =  0.95 DT =  25080.719553072624
dsnb SNOlab low alpha =  0.1 power =  0.95 DT =  13793.638026371756
dsnb SNOlab low alpha =  0.001 power =  0.8 DT =  24175.32517793594
dsnb SNOlab low alpha =  0.01 power =  0.8 DT =  15777.748375977728
dsnb SNOlab low alpha =  0.1 power =  0.8 DT =  7285.762987613238


In [30]:
sel = [col for col in result_pd.columns if 'hep' in col]
sel

['hep_CJPL_high',
 'hep_CJPL_low',
 'hep_Kamioka_high',
 'hep_Kamioka_low',
 'hep_LNGS_high',
 'hep_LNGS_low',
 'hep_SURF_high',
 'hep_SURF_low',
 'hep_SNOlab_high',
 'hep_SNOlab_low']

In [31]:
result_pd[sel]

,hep_CJPL_high,hep_CJPL_low,hep_Kamioka_high,hep_Kamioka_low,hep_LNGS_high,hep_LNGS_low,hep_SURF_high,hep_SURF_low,hep_SNOlab_high,hep_SNOlab_low
"(0.001, 0.9)",6725.789497,5187.650697,6723.320201,5192.099821,6735.737764,5195.700735,6740.943873,5188.917139,6737.887822,5194.42144
"(0.01, 0.9)",4595.293552,3542.117647,4587.08749,3535.383998,4583.268366,3538.788468,4588.893,3536.491208,4579.88558,3536.771074
"(0.1, 0.9)",2318.429213,1788.358417,2322.784173,1794.374782,2311.267343,1785.764989,2320.974746,1788.120215,2314.822153,1787.513091
"(0.001, 0.95)",7917.283627,6098.208768,7901.423571,6099.863433,7889.479659,6104.203477,7907.077839,6099.838181,7901.831812,6097.172462
"(0.01, 0.95)",5566.158808,4298.709271,5562.823037,4275.831066,5556.677764,4283.915658,5567.307386,4286.698597,5562.017873,4282.560916
"(0.1, 0.95)",3027.788235,2325.614861,3026.012987,2339.47619,3018.922568,2339.870229,3029.751971,2336.373083,3021.312356,2333.705882
"(0.001, 0.8)",5448.675768,4202.393312,5444.300246,4193.285847,5445.370933,4197.898721,5448.499461,4207.165794,5446.574704,4200.597498
"(0.01, 0.8)",3538.59515,2721.822385,3535.214471,2724.322115,3540.775922,2728.339589,3534.830986,2734.661623,3536.679355,2732.426338
"(0.1, 0.8)",1587.851462,1226.217857,1592.882409,1235.400381,1593.015569,1229.200477,1586.290897,1229.353383,1586.752842,1228.153529


In [18]:
result_pd

,atm_CJPL_high,atm_CJPL_low,atm_Kamioka_high,atm_Kamioka_low,atm_LNGS_high,atm_LNGS_low,atm_SURF_high,atm_SURF_low,atm_SNOlab_high,atm_SNOlab_low,...,dsnb_CJPL_high,dsnb_CJPL_low,dsnb_Kamioka_high,dsnb_Kamioka_low,dsnb_LNGS_high,dsnb_LNGS_low,dsnb_SURF_high,dsnb_SURF_low,dsnb_SNOlab_high,dsnb_SNOlab_low
"(0.001, 0.9)",3512.001583,3509.366379,2362.063922,2469.000701,1515.276203,1514.467152,909.632157,909.125648,902.42871,901.729856,...,17628.827842,17573.036735,19756.756318,20058.02127,23745.916852,23845.469531,29703.497525,29891.678073,29824.693145,30010.051202
"(0.01, 0.9)",2505.97153,2503.972973,1696.40081,1696.282473,1066.642587,1064.265424,641.643506,640.625363,637.645401,636.976972,...,12018.633555,12183.332954,13815.098706,13958.047952,16236.941099,16198.045359,20625.873406,20582.022638,20412.640147,20362.022893
"(0.1, 0.9)",1311.963749,1311.244352,1004.805339,1004.116417,594.599859,594.804304,360.940884,361.152625,358.62634,358.668524,...,6288.008283,6269.8857,7016.634615,7182.9699,8436.569779,8416.782837,10529.728324,10506.133333,10484.257293,10592.438415
"(0.001, 0.95)",4207.725579,4204.901408,2885.615556,2883.154828,1866.611384,1865.750848,1097.249144,1096.649389,1091.195631,1089.488908,...,20739.313278,20879.623056,23709.167379,23636.485459,27602.748028,28158.742615,32000.0,32000.0,32000.0,32000.0
"(0.01, 0.95)",3024.964869,3027.944899,2086.799645,2086.762695,1326.176944,1326.08422,766.477469,767.0124,761.718112,761.589298,...,14852.169471,14803.629187,16595.156217,16920.207509,19863.329873,19806.80468,24727.190603,24942.280333,24880.18244,24950.930114
"(0.1, 0.95)",1797.372671,1796.548919,1253.213632,1252.35631,761.299392,758.936364,475.164579,473.859694,471.826884,471.90661,...,8200.617886,8387.13628,9343.75453,9507.292359,11001.795572,10975.123192,13612.868547,13978.688806,13950.846851,13792.983976
"(0.001, 0.8)",2806.29546,2805.186756,1941.284674,1939.344508,1163.331761,1162.317348,718.029599,717.683226,713.564884,713.707259,...,14102.220246,14059.666667,15962.86926,16091.106965,18779.975,18890.254485,23958.972973,23908.267846,23979.565252,24057.110087
"(0.01, 0.8)",1839.582796,1837.578965,1302.358647,1301.581234,803.022186,803.138633,513.890705,513.534386,458.898642,458.566368,...,9191.990875,9358.801887,10474.115406,10446.494246,12305.860552,12429.609911,15627.078947,15721.967873,15812.350141,15776.480971
"(0.1, 0.8)",984.320329,984.257053,641.015367,640.517538,431.167214,431.107315,249.211893,248.796668,247.464447,247.425638,...,6000.0,6000.0,6000.0,6000.0,6052.555926,6037.498277,7199.686346,7185.572762,7168.741755,7153.198101


In [12]:
result_pd

,atm_CJPL_high,atm_CJPL_low,atm_Kamioka_high,atm_Kamioka_low,atm_LNGS_high,atm_LNGS_low,atm_SURF_high,atm_SURF_low,atm_SNOlab_high,atm_SNOlab_low,...,dsnb_CJPL_high,dsnb_CJPL_low,dsnb_Kamioka_high,dsnb_Kamioka_low,dsnb_LNGS_high,dsnb_LNGS_low,dsnb_SURF_high,dsnb_SURF_low,dsnb_SNOlab_high,dsnb_SNOlab_low
"(0.001, 0.9)",3503.499280,3506.303876,2362.853878,2470.731870,1514.438044,1513.073987,909.036716,908.479959,903.603100,902.340689,...,17628.024532,17571.499833,19937.936498,20058.881766,23751.448674,23842.097084,29703.261246,29895.408360,30080.316691,30012.988439
"(0.01, 0.9)",2505.372667,2508.087243,1697.776612,1696.511788,1064.763850,1065.255777,641.502053,640.844506,637.719307,637.424787,...,12027.502573,12188.385836,13806.375904,13957.108567,16236.157303,16199.876510,20626.824924,20578.564161,20415.088585,20496.342662
"(0.1, 0.9)",1311.493233,1310.688477,1004.838066,1004.446919,594.523596,594.870154,360.855188,360.524375,358.649208,358.521786,...,6291.972281,6270.298701,7021.269254,7185.586989,8439.055794,8415.833503,10533.267323,10506.235142,10484.199418,10592.190083


In [32]:
result_pd.to_csv(file_name)